In [1]:
import numpy as np
import torch
from preprocess.dataset import get_MNIST,get_dataset,get_handler
from models.model import Model
from al_methods.least_confidence import LeastConfidence
from ssl_methods.semi_fixmatch import fixmatch
import models
from torchvision import transforms
from framework.framework2 import Framework2
import torch.nn as nn
import time
from torch.utils.data import DataLoader
from scipy.spatial.distance import jensenshannon

In [2]:
# in the main 
X_tr, Y_tr, X_te, Y_te = get_dataset("Mnist", "./datasets")

In [3]:
# in the main 
if type(X_tr) is list:
    X_tr = np.array(X_tr)
    Y_tr = torch.tensor(np.array(Y_tr))
    X_te = np.array(X_te)
    Y_te = torch.tensor(np.array(Y_te))

if type(X_tr[0]) is not np.ndarray:
    X_tr = X_tr.numpy()
    X_te = X_te.numpy()
    

In [4]:
# in the main
n_pool = len(Y_tr)
n_test = len(Y_te)
#in the main
handler = get_handler("mnist")
# main or framewrok to see
nEnd=50 # total number to query 
nQuery=1 # nombre of points to query in batch 
nStart=10 # nbre of points to start
NUM_INIT_LB = int(nStart*n_pool/100)
NUM_QUERY = int(nQuery*n_pool/100) if nStart!= 100 else 0
NUM_ROUND = int((int(nEnd*n_pool/100) - NUM_INIT_LB)/ NUM_QUERY) if nStart!= 100 else 0
if NUM_QUERY != 0:
    if (int(nEnd*n_pool/100) - NUM_INIT_LB)% NUM_QUERY != 0:
        NUM_ROUND += 1
print(NUM_INIT_LB,NUM_ROUND,NUM_QUERY)
model_name = sorted(name for name in models.__dict__ 
                    if callable(models.__dict__[name]))
model_name
model=Model('resnet50').get_model()
model
model.conv1 = torch.nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)  # Conversion en 3 canaux
model.fc = torch.nn.Linear(2048, 10)  # Modifier la couche de classification pour 10 classes
# in the main file
idxs_lb = np.zeros(n_pool, dtype=bool)
idxs_lb
# in the main file
idxs_tmp = np.arange(n_pool)
idxs_tmp
np.random.shuffle(idxs_tmp)
idxs_tmp
# in the main file
idxs_lb[idxs_tmp[:NUM_INIT_LB]] = True
idxs_lb

6000 40 600


array([False, False, False, ..., False, False, False])

In [5]:
len(idxs_lb)

60000

In [6]:
len(idxs_lb[idxs_lb==True])

6000

In [7]:
args_pool = {'mnist':
                { 
                 'n_class':10,
                 'channels':1,
                 'size': 28,
                 'transform_tr': transforms.Compose([
                                transforms.RandomHorizontalFlip(),
                                transforms.ToTensor(), 
                                transforms.Normalize((0.1307,), (0.3081,))]),
                 'transform_te': transforms.Compose([transforms.ToTensor(), 
                                transforms.Normalize((0.1307,), (0.3081,))]),
                 'loader_tr_args':{'batch_size': 128, 'num_workers': 8},
                 'loader_te_args':{'batch_size': 1024, 'num_workers': 8},
                 'normalize':{'mean': (0.1307,), 'std': (0.3081,)},
                },
            
        }

In [8]:
class Args:
    def __init__(self,n_class,img_size,channels,transform_tr,transform_te,loader_tr_args,loader_te_args,normalize):
        self.n_class=n_class
        self.img_size=img_size
        self.channels=channels
        self.transform_tr=transform_tr
        self.transform_te=transform_te
        self.loader_tr_args=loader_tr_args
        self.loader_te_args=loader_te_args
        self.normalize=normalize
        self.dataset='mnist'
        self.save_path='./save'
        self.model='ResNet50'
        self.lr=0.1
        self.schedule = [20, 40]
        self.momentum=0.9
        self.gammas=[0.1,0.1]
        self.strategy='framwork1'
        self.optimizer='SGD'
        self.save_model=False

In [9]:
dataset_args = args_pool["mnist"]
n_class = dataset_args['n_class']
img_size = dataset_args['size']
channels = dataset_args['channels']
transform_tr = dataset_args['transform_tr']
transform_te = dataset_args['transform_te']
loader_tr_args = dataset_args['loader_tr_args']
loader_te_args = dataset_args['loader_te_args']
normalize = dataset_args['normalize']

In [10]:
args=Args(n_class,img_size,channels,transform_tr,transform_te,loader_tr_args,loader_te_args,normalize)

In [11]:
# al_strat=RandomSampling(X_tr, Y_tr, X_te, Y_te, idxs_lb, model, handler, args)
# ssl_strat=fixmatch(X_tr, Y_tr, X_te, Y_te, idxs_lb, model, handler, args)

In [12]:
framework_2= Framework2(X_tr, Y_tr, X_te, Y_te, idxs_lb, model, handler, args,LeastConfidence,fixmatch)

In [13]:
framework_2.train(2e-3,10)

Let's use 1 GPUs!
[Batch=000] [Loss=2.69]
[Batch=010] [Loss=3.87]
[Batch=020] [Loss=18.17]
[Batch=030] [Loss=7.31]
[Batch=040] [Loss=23.92]

==>>[2023-08-21 15:42:11] [Epoch=000/010] [framwork1 Need: 00:00:00] [LR=0.1000] [Best : Test Accuracy=0.00, Error=1.00]
[Batch=000] [Loss=11.89]
[Batch=010] [Loss=8.08]
[Batch=020] [Loss=2.21]
[Batch=030] [Loss=2.21]
[Batch=040] [Loss=4.57]

==>>[2023-08-21 15:42:14] [Epoch=001/010] [framwork1 Need: 00:00:32] [LR=0.1000] [Best : Test Accuracy=0.15, Error=0.85]
[Batch=000] [Loss=12.04]
[Batch=010] [Loss=2.84]
[Batch=020] [Loss=1.94]
[Batch=030] [Loss=4.74]
[Batch=040] [Loss=3.58]

==>>[2023-08-21 15:42:17] [Epoch=002/010] [framwork1 Need: 00:00:25] [LR=0.1000] [Best : Test Accuracy=0.16, Error=0.84]
[Batch=000] [Loss=2.65]
[Batch=010] [Loss=3.72]
[Batch=020] [Loss=1.75]
[Batch=030] [Loss=1.76]
[Batch=040] [Loss=1.74]

==>>[2023-08-21 15:42:20] [Epoch=003/010] [framwork1 Need: 00:00:21] [LR=0.1000] [Best : Test Accuracy=0.26, Error=0.74]
[Batch=000

FileNotFoundError: [Errno 2] No such file or directory: './save/mnist.png'

## Data augementation

In [ ]:
# # Create a weakly data augmentation transform
# weak_transform = transforms.Compose([
#     transforms.RandomCrop(14),
#     transforms.RandomHorizontalFlip(),
#     transforms.ToTensor()
# ])

# # Create a strongly data augmentation transform
# strong_transform = transforms.Compose([
#     transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
#     transforms.GaussianBlur(kernel_size=(5, 5), sigma=(0.1, 0.1)),
#     transforms.RandomPerspective(),
#     transforms.ToTensor()
# ])


In [ ]:
# # Find indices where the boolean matrix is False
# false_indices = np.argwhere(framework_2.idxs_lb == False).squeeze()

In [ ]:
# U_X_tr=X_tr[false_indices]
# U_Y_tr=Y_tr[false_indices]

In [ ]:
# unlabeled_data_weak=DataLoader(handler(U_X_tr,U_Y_tr,transform=weak_transform))
# unlabeled_data_strong=DataLoader(handler(U_X_tr,U_Y_tr,transform=strong_transform))

### weakly augmented

In [ ]:
# net=framework_2.net.eval()
# predictions_weak = []
# with torch.no_grad():
#     for x, _,_ in unlabeled_data_weak:
#         # Move the batch to the appropriate device (CPU or GPU)
#         x= x.to(framework_2.device)  # device could be 'cuda' or 'cpu'
        
#         # Forward pass to obtain predictions
#         batch_predictions = net(x)
        
#         # Append batch predictions to the list
#         predictions_weak.append(batch_predictions.cpu().numpy())  # Move predictions to CPU and convert to numpy

# # Concatenate predictions from all batches
# predictions_weak = np.concatenate(predictions_weak, axis=0)


In [ ]:
# net=framework_2.net.eval()
# predictions_strong = []
# with torch.no_grad():
#     for x, _,_ in unlabeled_data_strong:
#         # Move the batch to the appropriate device (CPU or GPU)
#         x= x.to(framework_2.device)  # device could be 'cuda' or 'cpu'
        
#         # Forward pass to obtain predictions
#         batch_predictions = net(x)
        
#         # Append batch predictions to the list
#         predictions_strong.append(batch_predictions.cpu().numpy())  # Move predictions to CPU and convert to numpy

# # Concatenate predictions from all batches
# predictions_strong = np.concatenate(predictions_strong, axis=0)


In [ ]:
# jensen=jensenshannon(predictions_weak,predictions_strong,axis=1)

In [ ]:
# np.max(predictions_weak,axis=1)

In [ ]:
# 0.5*(np.max(predictions_weak,axis=1)+np.max(predictions_strong,axis=1))

In [ ]:
# tot=(1-jensen ) *0.5*(np.max(predictions_weak,axis=1)+np.max(predictions_strong,axis=1))

In [ ]:
test_acc=framework_2.predict(X_te,Y_te)

In [ ]:
test_acc

In [ ]:
acc = np.zeros(NUM_ROUND+1)
acc[0] = test_acc

In [ ]:
len(acc)

In [ ]:
for rd in range(1, NUM_ROUND+1):

    if rd%2==0:
        # Al_methods
        print('Round {}/{}'.format(rd, NUM_ROUND), flush=True)
        labeled = len(np.arange(n_pool)[idxs_lb])
        if NUM_QUERY > int(nEnd*n_pool/100) - labeled:
            NUM_QUERY = int(nEnd*n_pool/100) - labeled
            
        # query
        ts = time.time()
        output = framework_2.stratAl.query(NUM_QUERY)
        q_idxs = output
        idxs_lb[q_idxs] = True
        te = time.time()
        tp = te - ts
        
        # update
        framework_2.update(idxs_lb)
        best_test_acc = framework_2.train(alpha=2e-3, n_epoch=10)

        t_iter = time.time() - ts
        
        # round accuracy
        # test_acc = strategy.predict(X_te, Y_te)
        acc[rd] = best_test_acc
    else:
        #SSL methods
        
        print('Round {}/{}'.format(rd, NUM_ROUND), flush=True)
        labeled = len(np.arange(n_pool)[idxs_lb])
        if NUM_QUERY > int(nEnd*n_pool/100) - labeled:
            NUM_QUERY = int(nEnd*n_pool/100) - labeled
            
        # query
        ts = time.time()

        output = framework_2.stratSSL.query(NUM_QUERY)
        q_idxs = output
        idxs_lb[q_idxs] = True
        te = time.time()
        tp = te - ts
        
        # update
        framework_2.update(idxs_lb)
        best_test_acc = framework_2.stratSSL.train(alpha=2e-3, n_epoch=10)

        t_iter = time.time() - ts
        
        # round accuracy
        # test_acc = strategy.predict(X_te, Y_te)
        acc[rd] = best_test_acc


In [ ]:
acc

In [ ]:
np.save("least_fix_acc.npy",acc)

In [ ]:
import gc

In [ ]:
gc.collect()